In [1]:
suppressMessages({
    require(plyr)
    require(tidyverse)
    require(data.table)
    require(readxl)
    require(viridis)
    require(ggplot2)
    require(yaml)
    require(cowplot)
    require(lubridate)
})

options(repr.plot.res=224)

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'viridis'"


In [2]:
dt.england <- readRDS(file="data/dt.england.rds")
dt.scotland <- readRDS(file="data/dt.scotland.rds")
dt.all <- rbind(dt.england, dt.scotland)
dt.all[, region := factor(region)]
str(dt.all)

Classes 'data.table' and 'data.frame':	260 obs. of  23 variables:
 $ date_range                              : chr  "03 August 2020 to 09 August 2020" "07 August 2020 to 13 August 2020" "14 August 2020 to 20 August 2020" "19 August 2020 to 25 August 2020" ...
 $ est_pct_positive                        : num  0.05 0.05 0.05 0.05 0.07 0.11 0.19 0.21 0.41 0.62 ...
 $ est_pct_positive_ci95_lo                : num  0.03 0.03 0.04 0.04 0.05 0.09 0.16 0.18 0.37 0.57 ...
 $ est_pct_positive_ci95_hi                : num  0.07 0.06 0.07 0.07 0.1 0.14 0.23 0.24 0.45 0.66 ...
 $ est_pop_positive                        : num  28300 24600 28200 27100 39700 ...
 $ est_pop_positive_ci95_lo                : num  19000 16900 20100 19300 29300 ...
 $ est_pop_positive_ci95_hi                : num  40700 33800 37900 36700 52700 ...
 $ est_ratio_positive                      : chr  "1 in 2,000" "1 in 2,200" "1 in 1,900" "1 in 2,000" ...
 $ est_ratio_positive_ci95_lo              : chr  "1 in 3,300" "1 in 3,

In [3]:
date_study_start <- as.Date('2021-07-01')
date_study_end <- as.Date('2022-12-16')
date_omicron_takeover <- as.Date("2021-12-14")
last_ons_date <- max(dt.all$date_hi)
last_ons_date

[1] "2022-12-09"

In [4]:
dt.asx_test_stopped <- data.table(
    region = factor(c("England", "Scotland")),
    stop_date = as.Date(c("2022-08-31", "2022-09-28"))
)
stop_dates <- dt.asx_test_stopped$stop_date
names(stop_dates) <- dt.asx_test_stopped$region
dt.asx_test_stopped 

region,stop_date
<fct>,<date>
England,2022-08-31
Scotland,2022-09-28


In [5]:
dt.study_period <- dt.all[date >= date_study_start & date <= date_study_end, ]
nrow(dt.study_period)

[1] 153

In [6]:
dt.study_period[, omicron_dominant := date > date_omicron_takeover]
dt.study_period[, testing_stopped := date > stop_dates[region]]
dt.study_period[seq(1, nrow(dt.study_period), 10), .(date, region, omicron_dominant, testing_stopped)]

date,region,omicron_dominant,testing_stopped
<date>,<fct>,<lgl>,<lgl>
2021-07-02,England,FALSE,FALSE
2021-09-10,England,FALSE,FALSE
2021-11-19,England,FALSE,FALSE
2022-01-28,England,TRUE,FALSE
2022-04-08,England,TRUE,FALSE
2022-06-17,England,TRUE,FALSE
2022-08-26,England,TRUE,FALSE
2022-11-04,England,TRUE,TRUE
2021-07-25,Scotland,FALSE,FALSE


In [7]:
dt.study_period[
    ,
    .(
        sum.hospital_onset_adm_count = sum(hospital_onset_adm_count), 
        sum.community_onset_adm_count = sum(community_onset_adm_count),
        sum.total_covid_adm_count = sum(total_covid_adm_count),
        ratio.hoco = sum(hospital_onset_adm_count) / sum(community_onset_adm_count)
    ),
    by = "region"
]

region,sum.hospital_onset_adm_count,sum.community_onset_adm_count,sum.total_covid_adm_count,ratio.hoco
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
England,120115,398264,518379,0.3015964
Scotland,12335,34196,46531,0.3607147


In [8]:
dt.study_period[
    ,
    .(
        sum.hospital_onset_adm_count = sum(hospital_onset_adm_count), 
        sum.community_onset_adm_count = sum(community_onset_adm_count)
    ),
    by = c("region", "testing_stopped")
]

region,testing_stopped,sum.hospital_onset_adm_count,sum.community_onset_adm_count
<fct>,<lgl>,<dbl>,<dbl>
England,FALSE,92982,344044
England,TRUE,27133,54220
Scotland,FALSE,10297,31907
Scotland,TRUE,2038,2289


In [9]:
dt.study_period[
    ,
    .(
        mean.hospital_onset_vs_comm_ratio = round(mean(hospital_onset_vs_comm_ratio), digits=2), 
        sd.hospital_onset_vs_comm_ratio = round(sd(hospital_onset_vs_comm_ratio), digits=2)
    ),
    by = c("region", "omicron_dominant", "testing_stopped")
]

region,omicron_dominant,testing_stopped,mean.hospital_onset_vs_comm_ratio,sd.hospital_onset_vs_comm_ratio
<fct>,<lgl>,<lgl>,<dbl>,<dbl>
England,FALSE,FALSE,0.12,0.03
England,TRUE,FALSE,0.33,0.04
England,TRUE,TRUE,0.48,0.10
Scotland,FALSE,FALSE,0.11,0.06
Scotland,TRUE,FALSE,0.43,0.10
Scotland,TRUE,TRUE,0.89,0.18


In [10]:
dt.study_period[
    ,
    .(
        mean.hospital_onset_vs_comm_positives = round(mean(hospital_onset_vs_comm_positives) * 1000, digits=2), 
        sd.hospital_onset_vs_comm_positives = round(sd(hospital_onset_vs_comm_positives) * 1000, digits=2)
    ),
    by = c("region", "omicron_dominant", "testing_stopped")
]

region,omicron_dominant,testing_stopped,mean.hospital_onset_vs_comm_positives,sd.hospital_onset_vs_comm_positives
<fct>,<lgl>,<lgl>,<dbl>,<dbl>
England,FALSE,FALSE,0.64,0.14
England,TRUE,FALSE,1.00,0.17
England,TRUE,TRUE,1.39,0.34
Scotland,FALSE,FALSE,0.78,0.37
Scotland,TRUE,FALSE,0.99,0.21
Scotland,TRUE,TRUE,1.64,0.37
